### Перед вами файл с получением данных о запуске космических аппаратов с сайта Gunter (https://space.skyrocket.de./index.html)

* Для получения данных введите необходимые года через запятую

In [ ]:
years = [2021]

* Для выгрузки определенного месяца, введите номара месяцев через запятую

In [ ]:
months = [3]

* После введения года и месяца, нажмите ctrl+F9 для запуска программы

### .................................................................................................

### Оглавление <a class="anchor" id="home"></a>

* [Установка библиотек и проверка соеднения](#1-bullet)
* [Парсинг уникальных ссылок](#2-bullet)
* [Парсинг ТТХ КА и информации о запуске](#3-bullet)
* [Подготовка таблицы к выгрузке в базу данных](#4-bullet)
* [Сохранение](#5-bullet)
* [Архивные строки кода](#5-bullet)

### Установка библиотек и проверка соеднения <a class="anchor" id="1-bullet"></a>

[Обратно в оглавление](#home)

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import string
import datetime as dt

Проверка соединения с сайтом

In [ ]:
url = 'https://space.skyrocket.de/directories/chronology.htm'
response = requests.get(url)
if response.status_code == 200:
    print('Запрос прошел успешно')
else:
    print(f'Ошибка. Response code: {response.status_code}')
    print(f'URL: {url}')

Запрос прошел успешно


Смоделируем агента для парсинга

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

response = requests.get(url, headers=headers, allow_redirects=True)
page_text = ''
if response.status_code == 200:
    print('Запрос прошел успешно!')
    page_text = response.text
else:
    print(f'Ошибка! Response code: {response.status_code}')
    print(f'URL: {url}')
    print('--------------')

Запрос прошел успешно!


### Парсинг уникальных ссылок <a class="anchor" id="2-bullet"></a>

[Обратно в оглавление](#home)

Извлечем таблицу с годами пусков и ссылками на страницы

In [ ]:
page_bs = BeautifulSoup(page_text)
coverpage = page_bs.find('table', {'class': 'index'})


Ссылки для перехода по годам сохраним в переменной в словаре full_link. Ключ словаря - год пусков, значение - ссылка

In [ ]:
main_link = 'https://space.skyrocket.de'
full_link = {}
launch_year = []
all_launch_years = []
for link in coverpage.find_all('a'):
    launch_year = link.renderContents()
    full_link[int(launch_year)] = main_link + link.get('href')[2:]
    all_launch_years.append(int(launch_year))
print('Тестовая ссылка',full_link[1957])
print('Тестовый год',all_launch_years[0])

Тестовая ссылка https://space.skyrocket.de/doc_chr/lau1957.htm
Тестовый год 1957


Парсинг страницы с годом запуска

Сделаем функцию для парсинга уникальных ссылок на КА со страницы с годом запуска

In [ ]:
def launch_data (link_year):
    url_launch_year = link_year
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response_launch = requests.get(url_launch_year, headers=headers, allow_redirects=True)
    page_launch_year = response_launch.text
    page_bs_launch_year = BeautifulSoup(page_launch_year)
    coverpage_launch_year = page_bs_launch_year.find('table', {'id': 'chronlist'})
    main_link = 'https://space.skyrocket.de'
    payload_links = []
    error = 0
    for link in coverpage_launch_year.find_all('a'):
        try:
            if 'sdat' in link.get('href') and (main_link + link.get('href')[2:]) not in payload_links:
                payload_links.append(main_link + link.get('href')[2:])
        except TypeError:
            error +=1
    return payload_links

Соберем список всех уникальных ссылок на КА

In [ ]:
all_satellites_link_list = []
all_satellites_link = []
#test_years = [1996]
#for year in all_launch_years:
#for year in test_years:
for year in years:
    all_satellites_link_list.append(launch_data(full_link[year]))
for lst in all_satellites_link_list:
    all_satellites_link.extend(lst)
unique_satellites_link = []
for satellite_link in all_satellites_link:
    if satellite_link not in unique_satellites_link:
        unique_satellites_link.append(satellite_link)

#np.unique(all_satellites_link_list)
#print(len(all_satellites_link))
#print(len(unique_satellites_link))
#print(all_satellites_link)
#print(unique_satellites_link)


In [ ]:
print('Количество уникальных КА',len(unique_satellites_link))

Количество уникальных КА 32


### Парсинг ТТХ КА и информации о запуске <a class="anchor" id="3-bullet"></a>

[Обратно в оглавление](#home)

#### Пасринг ТТХ КА
* Открываем каждую уникальную ссылку КА
* На каждой странице берем название КА - заголовок h1.
* ТТХ КА ('table', {'id': 'satdata'})
* Делаем словарь. Первый ключ - columns, значения - названия столбцов таблицы ('th', {'class': 'lhead'}). Далее ключи - навание КА, значения список характеристик из таблицы ('td', {'class': 'rcont'})
* Делаем dataframe с первой строкой - наименования столбцов, дальше добавляем КА циклом из словаря

#### Парсинг информации о запуске КА

* Открываем каждую уникальную ссылку КА
* На каждой странице берем название КА - заголовок h1.
* Данные о запуске КА ('table', {'id': 'satlist'})

#### Получение названия колонок

Получим название колонок с ТТХ КА

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response_satellite = requests.get(unique_satellites_link[0], headers=headers, allow_redirects=True)
page_bs_launch_year = BeautifulSoup(response_satellite.text)
    
satellite_data_title = ['satellite_name']
coverpage_characteristics = page_bs_launch_year.find('table', {'id': 'satdata'})
satellite_data_coltitle = coverpage_characteristics.find_all('th', {'class': 'lhead'})

for column in range(len(satellite_data_coltitle)):
    satellite_data_title.append(str(satellite_data_coltitle[column].renderContents())[2:-1])

#satellite_data_title

Получим название колонок в таблице о запуске КА

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response_satellite = requests.get(unique_satellites_link[0], headers=headers, allow_redirects=True)
page_bs_launch_year = BeautifulSoup(response_satellite.text)

coverpage_characteristics = page_bs_launch_year.find('table', {'id': 'satlist'})
satellite_data_coltitle = coverpage_characteristics.find_all('th')
#satellite_launch_columns = ['satellite_name']
satellite_launch_columns = []
for columns in satellite_data_coltitle:
    satellite_launch_columns.append(str(columns.renderContents())[2:-1])
satellite_launch_columns.append('satellite_name')

#satellite_launch_columns

#### Создание функций для парсинга

Сделаем функцию по парсингу данных из таблицы с характеристиками КА

In [ ]:
def satellite_characteristic (link, final_dict):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response_satellite = requests.get(link, headers=headers, allow_redirects=True)
    page_bs_launch_year = BeautifulSoup(response_satellite.text)
    satellite_name = str(page_bs_launch_year.find('h1').renderContents().decode('utf-8','ignore'))
    satellite_data_value = [satellite_name]
    coverpage_characteristics = page_bs_launch_year.find('table', {'id': 'satdata'})
    satellite_data_colvalue = coverpage_characteristics.find_all('td', {'class': 'rcont'})
    #
    #satellite_data_colvalue=satellite_data_colvalue.decode('utf-8','ignore').encode("utf-8")
    #
    for column in range(len(satellite_data_colvalue)):
        satellite_data_value.append(str(satellite_data_colvalue[column].renderContents().decode('utf-8','ignore')))
    final_dict[satellite_name] = satellite_data_value
    return final_dict

Подготовим таблицу со столбцами запуска КА для будущего парсинга 

In [ ]:
launch_data = pd.DataFrame(columns = satellite_launch_columns)
display(launch_data)

,Satellite,COSPAR,Date,LS,,Launch Vehicle,Remarks,satellite_name


Сделаем функцию по получению данных о запуске КА

In [ ]:
def launch_characteristic (link, dataframe):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response_satellite = requests.get(link, headers=headers, allow_redirects=True)
    page_bs_launch_year = BeautifulSoup(response_satellite.text)
    satellite_name = str(page_bs_launch_year.find('h1').renderContents().decode('utf-8','ignore'))
    coverpage_characteristics = page_bs_launch_year.find('table', {'id': 'satlist'})
    satellite_data_values = coverpage_characteristics.find_all('td')
    string_length = len(satellite_launch_columns)-1
    strings = int(len(satellite_data_values)/string_length)
    satellite_data_context = []
    #делаем датафрейм
    satellite_data = [satellite_name]    
    for element in satellite_data_values: 
        value = str(element.renderContents().decode('utf-8','ignore'))
        satellite_data_context.append(value) 
        #satellite_data_context.append(str(element.renderContents())[2:-1])  
        #display(str(element.renderContents())[2:-1])
    for string in range(strings):
        satellite_data = satellite_data_context[string*string_length:(string+1)*string_length]
        satellite_data.append(satellite_name)
        dataframe.loc[len(dataframe)+1] = satellite_data
    return dataframe

#### Парсим данные по харакетирстикам и запуску КА

* Открываем уникальную страницу
* На каждой странице парсим таблицу с харакетиристиками с помощью функции satellite_characteristic и таблицу с запусками с помощью функции launch_characteristic

In [ ]:
satellites_characteristics = {}
for satellite_link in unique_satellites_link:
    satellite_characteristic (satellite_link, satellites_characteristics)
    launch_characteristic(satellite_link,launch_data)

#display(launch_data)
#display(satellites_characteristics)

Делаем таблицу с харакетиристиками КА

In [ ]:
characteristics_data = pd.DataFrame(columns = satellite_data_title)
n = 0
for satellite in satellites_characteristics.keys():
    characteristics_data.loc[n] = satellites_characteristics[satellite]
    n+=1
data = characteristics_data.drop([0])
characteristics_data.head(10)

,satellite_name,Nation:,Type / Application:,Operator:,Contractors:,Equipment:,Configuration:,Propulsion:,Power:,Lifetime:,Mass:,Orbit:
0,Türksat 5A,Turkey,Communication,Türksat AS,"Airbus Defence and Space, Turkish Aerospace In...",Ku-band transponders,"<a href=""../doc_sat/astrium_eurostar-3000.htm""...",?,"2 deployable solar arrays, batteries",15 years,3500 kg,GEO
1,CACTUS 1,USA,Technology,Capitol Technology University,Capitol Technology University,,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,2.8 kg,
2,"ExoCube 1, 2 (CP 10, 12)",USA,"Science, atmosphere","NASA/JPL, Cal Poly Picosatellite Project (Poly...","NASA/JPL, Cal Poly Picosatellite Project (Poly...",,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,4 kg (#1); 3.2 kg (#2),"439 km × 669 km, 99.13°"
3,MiTEE 1,USA,Technology,University of Michigan,University of Michigan,Miniature electrodynamic tether system,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,3.4 kg,
4,PolarCube,USA,Technology,Colorado Space Grant Consortium (CoSGC),Colorado Space Grant Consortium (CoSGC),,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)<...",None,"Solar cells, batteries",1 year,3.9 kg,
5,"Q-PACE (Cu-PACE, SIMPLEx 2)",USA,Microgravity,University of Central Florida (UCF),University of Central Florida (UCF),Test cell,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,2.76 kg,
6,TechEdSat 7 (TES 7),USA,"Technology, education","SJSU, University of Idaho, NASA Ames Research ...","SJSU, University of Idaho, NASA Ames Research ...","High Packing-Density Exo-Brake, CUBIT","<a href=""../doc_sat/cubesat.htm"">CubeSat (2U)</a>",None,"Solar cells, batteries",,2.5 kg,
7,CAPE 3,USA,Technology,University of Louisiana at Lafayette,University of Louisiana at Lafayette,,"<a href=""../doc_sat/cubesat.htm"">CubeSat (1U)</a>",None,"Solar cells, batteries",,1.3 kg,
8,"RadFxSat 2 (Fox 1E, Evolution)",USA,"Amateur communication, technology","AMSAT, Vanderbilt University",AMSAT,Mode J linear transponder,"<a href=""../doc_sat/cubesat.htm"">CubeSat (1U)</a>",None,"Solar cells, batteries",,1.3 kg,
9,"PICS 1, 2",USA,Technology,"Brigham Young University, Provo, Utah","Brigham Young University, Provo, Utah",,"<a href=""../doc_sat/cubesat.htm"">CubeSat (1U)</a>",,"Solar cells, batteries",,1.35 kg,


#### Объединим 2 таблицы 

Длина каждой таблицы

In [ ]:
print('Количество уникальных характеристик КА',len(characteristics_data))
print('Количество запущенных КА',len(launch_data))

Количество уникальных характеристик КА 32
Количество запущенных КА 106


Объединение таблиц с помощью merge с параметр внешний (outer), чтобы не потерять данные

In [ ]:
satellite_data = pd.merge(characteristics_data,launch_data, on = 'satellite_name', how = 'outer')
#satellite_data = characteristics_data.join(launch_data, rsuffix = 'r', how = 'outer')
#satellite_data = pd.concat([characteristics_data,launch_data],axis = 1,  join = 'outer')
display(satellite_data.head(5))
print(len(satellite_data))

,satellite_name,Nation:,Type / Application:,Operator:,Contractors:,Equipment:,Configuration:,Propulsion:,Power:,Lifetime:,Mass:,Orbit:,Satellite,COSPAR,Date,LS,,Launch Vehicle,Remarks
0,Türksat 5A,Turkey,Communication,Türksat AS,"Airbus Defence and Space, Turkish Aerospace In...",Ku-band transponders,"<a href=""../doc_sat/astrium_eurostar-3000.htm""...",?,"2 deployable solar arrays, batteries",15 years,3500 kg,GEO,Türksat 5A,2021-001A,08.01.2021,CC SLC-40,,"<a href=""../doc_lau_det/falcon-9_v1-2_b5.htm"">...",
1,CACTUS 1,USA,Technology,Capitol Technology University,Capitol Technology University,,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,2.8 kg,,CACTUS 1,2021-002,17.01.2021,Mo RW12/30,,"<a href=""../doc_lau/launcherone.htm"">LauncherO...","with <a href=""exocube.htm"">ExoCube 2</a>, <a h..."
2,"ExoCube 1, 2 (CP 10, 12)",USA,"Science, atmosphere","NASA/JPL, Cal Poly Picosatellite Project (Poly...","NASA/JPL, Cal Poly Picosatellite Project (Poly...",,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,4 kg (#1); 3.2 kg (#2),"439 km × 669 km, 99.13°",ExoCube (CP 10),2015-003E,31.01.2015,Va SLC-2W,,"<a href=""../doc_lau_det/delta-7320.htm"">Delta-...","with <a href=""smap.htm"">SMAP</a>, <a href=""gri..."
3,"ExoCube 1, 2 (CP 10, 12)",USA,"Science, atmosphere","NASA/JPL, Cal Poly Picosatellite Project (Poly...","NASA/JPL, Cal Poly Picosatellite Project (Poly...",,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,4 kg (#1); 3.2 kg (#2),"439 km × 669 km, 99.13°",ExoCube 2 (CP 12),2021-002L,17.01.2021,Mo RW12/30,,"<a href=""../doc_lau/launcherone.htm"">LauncherO...","with <a href=""cactus-1.htm"">CACTUS 1</a>, <a h..."
4,MiTEE 1,USA,Technology,University of Michigan,University of Michigan,Miniature electrodynamic tether system,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,3.4 kg,,MiTEE 1,2021-002F,17.01.2021,Mo RW12/30,,"<a href=""../doc_lau/launcherone.htm"">LauncherO...","with <a href=""cactus-1.htm"">CACTUS 1</a>, <a h..."


106


Добавление столбца с годом и месяцем запуска, изменение формата даты пуска 

In [ ]:
#satellite_data.info()

satellite_data['Date'] = pd.to_datetime(satellite_data['Date'], format = '%d.%m.%Y',errors='coerce')
satellite_data['year'] = satellite_data['Date'].dt.year
satellite_data['month'] = satellite_data['Date'].dt.month
#satellite_data['Date'] = {0}/{1}/{2:02}'.format(satellite_data['Date'].month, satellite_data['Date'].day, satellite_data['Date'].year % 100)
errors = 0
for string in range(len(satellite_data)):
    try:
        satellite_data.loc[string,'Date'] = dt.datetime.strptime(str(satellite_data.loc[string,'Date']), '%Y-%m-%d %H:%M:%S').strftime('%d.%m.%Y')
    except ValueError:
        errors+=1
    #satellite_data.loc[string,'Date'] = {0}/{1}/{2:02}'.format(dt.month, dt.day, dt.year % 100)
print(errors)
display(satellite_data.head(5))

39


,satellite_name,Nation:,Type / Application:,Operator:,Contractors:,Equipment:,Configuration:,Propulsion:,Power:,Lifetime:,Mass:,Orbit:,Satellite,COSPAR,Date,LS,,Launch Vehicle,Remarks,year,month
0,Türksat 5A,Turkey,Communication,Türksat AS,"Airbus Defence and Space, Turkish Aerospace In...",Ku-band transponders,"<a href=""../doc_sat/astrium_eurostar-3000.htm""...",?,"2 deployable solar arrays, batteries",15 years,3500 kg,GEO,Türksat 5A,2021-001A,08.01.2021,CC SLC-40,,"<a href=""../doc_lau_det/falcon-9_v1-2_b5.htm"">...",,2021.0,1.0
1,CACTUS 1,USA,Technology,Capitol Technology University,Capitol Technology University,,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,2.8 kg,,CACTUS 1,2021-002,17.01.2021,Mo RW12/30,,"<a href=""../doc_lau/launcherone.htm"">LauncherO...","with <a href=""exocube.htm"">ExoCube 2</a>, <a h...",2021.0,1.0
2,"ExoCube 1, 2 (CP 10, 12)",USA,"Science, atmosphere","NASA/JPL, Cal Poly Picosatellite Project (Poly...","NASA/JPL, Cal Poly Picosatellite Project (Poly...",,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,4 kg (#1); 3.2 kg (#2),"439 km × 669 km, 99.13°",ExoCube (CP 10),2015-003E,31.01.2015,Va SLC-2W,,"<a href=""../doc_lau_det/delta-7320.htm"">Delta-...","with <a href=""smap.htm"">SMAP</a>, <a href=""gri...",2015.0,1.0
3,"ExoCube 1, 2 (CP 10, 12)",USA,"Science, atmosphere","NASA/JPL, Cal Poly Picosatellite Project (Poly...","NASA/JPL, Cal Poly Picosatellite Project (Poly...",,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,4 kg (#1); 3.2 kg (#2),"439 km × 669 km, 99.13°",ExoCube 2 (CP 12),2021-002L,17.01.2021,Mo RW12/30,,"<a href=""../doc_lau/launcherone.htm"">LauncherO...","with <a href=""cactus-1.htm"">CACTUS 1</a>, <a h...",2021.0,1.0
4,MiTEE 1,USA,Technology,University of Michigan,University of Michigan,Miniature electrodynamic tether system,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",None,"Solar cells, batteries",,3.4 kg,,MiTEE 1,2021-002F,17.01.2021,Mo RW12/30,,"<a href=""../doc_lau/launcherone.htm"">LauncherO...","with <a href=""cactus-1.htm"">CACTUS 1</a>, <a h...",2021.0,1.0


### Подготовка таблицы к выгрузке в базу данных <a class="anchor" id="4-bullet"></a>

[Обратно в оглавление](#home)

Делаем срез таблицы согласно изначальным условиям по году и месяцу

In [ ]:
#year = 1982
#month = 11
data_slice = satellite_data.query('year == @years & month == @months')
data_slice.reset_index(drop=False, inplace = True)
display(data_slice)

,index,satellite_name,Nation:,Type / Application:,Operator:,Contractors:,Equipment:,Configuration:,Propulsion:,Power:,Lifetime:,Mass:,Orbit:,Satellite,COSPAR,Date,LS,,Launch Vehicle,Remarks,year,month
0,92,"Hiber 3, ..., x",Netherlands,Communication M2M/IoT,Hiber Global (ex Magnitude Space),,,"<a href=""../doc_sat/cubesat.htm"">CubeSat (3U)</a>",?,"Solar cells, batteries",,,,Hiber 3,2021-022L,22.03.2021,Ba LC-31/6,,"<a href=""../doc_lau_det/soyuz-2-1a_fregat.htm""...","with <a href=""cas500-1.htm"">CAS500 1</a>, <a h...",2021.0,3.0


### Очистка данных

Функция по удалению тегов ссылок 

In [ ]:
def replace (data,string,column):
    element = str(data.loc[string,column])   
    a_1 = element.find('<a',0,len(element))
    a_2 = element.find('>',0,len(element))
    replace_1 = element[int(a_1):int(a_2)+1]
    #display(replace_1)
    a_3 = element.find('</a',a_2+1,len(element))
    a_4 = element.find('>',a_2+1,len(element))
    replace_2 = element[int(a_3):int(a_4)+1]
    #display(replace_2)
    element = element.replace(replace_1,'')
    element = element.replace(replace_2,'')
    return element

Замена тегов ссылок

In [ ]:
replace_coulmns = ['Satellite','LS','Launch Vehicle','Remarks']
for column in satellite_launch_columns:
#for column in replace_coulmns:    
    for string in data_slice.index:
        data_slice.loc[string,column] = replace(data_slice,string,column)
        
for string in data_slice.index:
    data_slice.loc[string,'Remarks'] = replace(data_slice,string,'Remarks')
for string in data_slice.index:
    data_slice.loc[string,'Configuration:'] = replace(data_slice,string,'Configuration:')
for string in data_slice.index:
    data_slice.loc[string,'Propulsion:'] = replace(data_slice,string,'Propulsion:')
for string in data_slice.index:
    data_slice.loc[string,'Propulsion:'] = replace(data_slice,string,'Power:')

#display(data_slice.head(5))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Добавим порядковый номер пуска. Порядковый номер - 

#### Добавление необходимых столбцов и подоговка к выгрузке в базу данных

Название колонок в базе данных

In [ ]:
#data_base_columns = pd.read_excel('data_base_columns.xlsx')
#data_base_columns = data_base_columns.columns


data_base_columns = \
['Year',\
 'Launch Date',\
 'Launch ID',\
 'Launched/Planned',\
 'Spacecraft Name',\
 'Spacecraft Cost At New  (US$m)',\
 'Цена КА (найдена/выставлена по среднему, исходя из сегмента)',\
'Total Launch Cost (US$m)',\
'Constellation Group',\
'Spacecraft Operator Country',\
 'Spacecraft Operator',\
'Operator Type',\
'Spacecraft Prime Manufacturer (Original)',\
'Spacecraft Prime Manufacturer Country',\
'Spacecraft Prime Manufacturer Type',\
'Spacecraft Prime Manufacturer Category',\
'Доступный/недоступный для производства',\
'Sector',\
'Mission',\
'Mission 2',\
'Mission 3',\
'Mission 4',\
'Orbit Category',\
'Orbit Sub Category',\
'Longitude',\
'Perigee (km)',\
'Apogee (km)',\
'Inclination (deg)',\
'Period (mins)',\
'Mass At Launch (kg)',\
'ADD',\
'Primary/Secondary Payload',\
'Mass Dry (kg)',\
'Mass Category',\
'Spacecraft Category',\
'Expected Life (years)',\
'Decay date (fact)',\
'Overtime, years',\
'Planned decay year (inc. lifetime and overtime)',\
'Spacecraft Status',\
'Bus Type',\
'Bus Manufacturer (Original)',\
'Bus Manufacturer (Original) Country',\
'Наличие аппаратуры САРСАТ',\
'Launch Site Name',\
'Launch Site Country',\
'Vehicle Long Name',\
'Vehicle Category',\
'Vehicle Family',\
'Vehicle Type',\
'Vehicle Variant',\
'Счетчик по РН',\
'Vehicle Prime Manufacturer (Original)',\
'Vehicle Prime Manufacturer Type',\
'Vehicle Prime Manufacturer (Original) Country',\
'Launch Provider',\
'Launch Provider Country',\
'Вспомогательный для доступен/недоступен',\
'Доступный/недоступный запуск РН',\
'Launch Category',\
'International Number',\
'Catalogue Number (NORAD)',\
'Sucсess/Fail',\
'Статус для модели',\
'Satellite Number AGAT',\
'Сегменты для модели']


In [ ]:
display(data_slice.head(1))

,index,satellite_name,Nation:,Type / Application:,Operator:,Contractors:,Equipment:,Configuration:,Propulsion:,Power:,Lifetime:,Mass:,Orbit:,Satellite,COSPAR,Date,LS,,Launch Vehicle,Remarks,year,month
0,92,"Hiber 3, ..., x",Netherlands,Communication M2M/IoT,Hiber Global (ex Magnitude Space),,,CubeSat (3U),"Solar cells, batteries","Solar cells, batteries",,,,Hiber 3,2021-022L,22.03.2021,Ba LC-31/6,,Soyuz-2-1a Fregat,"with CAS500 1, ELSA-d Chaser, <a href=""elsa-d-...",2021.0,3.0


Создание таблицы в формате базы данных

In [ ]:
data_base = pd.DataFrame(columns = data_base_columns)
#display(data_base)


Добавление готовых столбцов (Year, Launch Date, Spacecraft Name, Spacecraft Operator Country

In [ ]:
data_base[data_base_columns[0]] = data_slice['year']
data_base[data_base_columns[1]] = data_slice['Date']
data_base[data_base_columns[3]] = 'Launched'
data_base[data_base_columns[4]] = data_slice['Satellite']
data_base[data_base_columns[9]] = data_slice['Nation:']
data_base[data_base_columns[10]] = data_slice['Operator:'] 
data_base[data_base_columns[12]] = data_slice['Contractors:'] #Производитель КА или полезной нагрузки?
data_base[data_base_columns[43]] = data_slice['Configuration:']
data_base[data_base_columns[49]] = data_slice['Launch Vehicle']
data_base[data_base_columns[62]] = data_slice['COSPAR']
data_base[data_base_columns[-3]] = 'Факт'

In [ ]:
mission_types = ['Ballast','Communications','Earth Observation','Navigation',\
                 'Other','Satellite servicing','Science','Space Station','Technology/Test','n/a']
#Заменим communication (из Gunter) на communications согласно классификаци базы
for string in data_slice.index:
    data_slice.loc[string,'Type / Application:'] = data_slice.loc[string,'Type / Application:'].\
    replace('Communication','Communications')
    if data_slice.loc[string,'Type / Application:'] in mission_types:
        data_slice.loc[string,'mission'] = data_slice.loc[string,'Type / Application:']

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
data_slice.head(5)

,index,satellite_name,Nation:,Type / Application:,Operator:,Contractors:,Equipment:,Configuration:,Propulsion:,Power:,Lifetime:,Mass:,Orbit:,Satellite,COSPAR,Date,LS,,Launch Vehicle,Remarks,year,month
0,92,"Hiber 3, ..., x",Netherlands,Communicationss M2M/IoT,Hiber Global (ex Magnitude Space),,,CubeSat (3U),"Solar cells, batteries","Solar cells, batteries",,,,Hiber 3,2021-022L,22.03.2021,Ba LC-31/6,,Soyuz-2-1a Fregat,"with CAS500 1, ELSA-d Chaser, <a href=""elsa-d-...",2021.0,3.0


In [ ]:
data_slice.groupby('mission')['mission'].count()

KeyError: ignored

Добавление столбца Launched/Planned

In [ ]:
for string in range(len(data_slice)):
    data_slice.loc[string,'Launched/Planned'] = 'Launched'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
display(data_base)

,Year,Launch Date,Launch ID,Launched/Planned,Spacecraft Name,Spacecraft Cost At New (US$m),"Цена КА (найдена/выставлена по среднему, исходя из сегмента)",Total Launch Cost (US$m),Constellation Group,Spacecraft Operator Country,Spacecraft Operator,Operator Type,Spacecraft Prime Manufacturer (Original),Spacecraft Prime Manufacturer Country,Spacecraft Prime Manufacturer Type,Spacecraft Prime Manufacturer Category,Доступный/недоступный для производства,Sector,Mission,Mission 2,Mission 3,Mission 4,Orbit Category,Orbit Sub Category,Longitude,Perigee (km),Apogee (km),Inclination (deg),Period (mins),Mass At Launch (kg),ADD,Primary/Secondary Payload,Mass Dry (kg),Mass Category,Spacecraft Category,Expected Life (years),Decay date (fact),"Overtime, years",Planned decay year (inc. lifetime and overtime),Spacecraft Status,Bus Type,Bus Manufacturer (Original),Bus Manufacturer (Original) Country,Наличие аппаратуры САРСАТ,Launch Site Name,Launch Site Country,Vehicle Long Name,Vehicle Category,Vehicle Family,Vehicle Type,Vehicle Variant,Счетчик по РН,Vehicle Prime Manufacturer (Original),Vehicle Prime Manufacturer Type,Vehicle Prime Manufacturer (Original) Country,Launch Provider,Launch Provider Country,Вспомогательный для доступен/недоступен,Доступный/недоступный запуск РН,Launch Category,International Number,Catalogue Number (NORAD),Sucсess/Fail,Статус для модели,Satellite Number AGAT,Сегменты для модели
0,2021.0,22.03.2021,NaN,Launched,Hiber 3,NaN,NaN,NaN,NaN,Netherlands,Hiber Global (ex Magnitude Space),NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CubeSat (3U),NaN,NaN,NaN,NaN,NaN,Soyuz-2-1a Fregat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-022L,Факт,NaN,NaN


### Сохранение <a class="anchor" id="5-bullet"></a>

[Обратно в оглавление](#home)

In [ ]:
data_base.to_excel('data_base.xls', index=False)
#data_slice.to_csv('data_slice.csv', index=False)

### Архивные строки кода <a class="anchor" id="6-bullet"></a>

[Обратно в оглавление](#home)

Добавим порядковый номер пуска

data_slice['launch_number'] = data_slice['COSPAR']
alphabet = string.ascii_uppercase
symbols = list(alphabet)
for symbol in symbols:
    #print(symbol)
    for string in range(len(data_slice)):
        data_slice.loc[string,'launch_number'] = data_slice.loc[string,'launch_number'].replace(symbol,'')
        #print(data_slice.loc[string,'launch_#'])

In [ ]:
data_slice.groupby('Date')['Date'].count().sort_values(ascending = True)

In [ ]:
data_slice.query('month == 10')

In [ ]:
display(data_slice.groupby('year')['year'].count())
display(data_slice.groupby('launch_number')['year'].count())

In [ ]:
data_slice.query('Date == "2020-03-07 00:00:00"')

In [ ]:
data_slice.query('launch_number == "1998-067"')

In [ ]:
data_slice.groupby('launch_#')['satellite_name'].count().sort_values(ascending = False).head(15)

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response_satellite = requests.get(link, headers=headers, allow_redirects=True)
    page_bs_launch_year = BeautifulSoup(response_satellite.text)
    satellite_name = str(page_bs_launch_year.find('h1').renderContents())[2:-1]    
    display(satellite_name)
    coverpage_characteristics = page_bs_launch_year.find('table', {'id': 'satlist'})
    satellite_data_values = coverpage_characteristics.find_all('td')
    string_length = len(satellite_launch_columns)
    strings = int(len(satellite_data_values)/string_length)
    satellite_data_context = []
    for element in satellite_data_values:
        satellite_data_context.append(str(element.renderContents())[2:-1])    
    for string in range(strings):
        launch_dict[satellite_name] = satellite_data_context[string*string_length:(string+1)*string_length-1]